# Load the tests to compare

In [1]:
import os
import sys
import json
import pickle
import pandas as pd
__file__ = globals()['_dh'][0]

# get an absolute path to the directory that contains parent files
project_dir = __file__ = globals()['_dh'][0]
sys.path.append(os.path.normpath(os.path.join(project_dir, '..', '..')))

from experiments.utils.constants import TESTS_RESULTS_PATH
series = 45
start_test = 0
end_test = 27
tests_ids = [i for i in range(start_test, end_test)]
tests_names = [i for i in range(start_test, end_test)]

tests = {}


for test_id, test_name in zip(tests_ids, tests_names):
    test_path = os.path.join(
        TESTS_RESULTS_PATH,
        'series',
        str(series),
        'tests',
        str(test_id))
    with open(os.path.join(test_path, 'info.json')) as cf:
        info = json.loads(cf.read())
    with open(os.path.join(test_path, 'episodes.pickle'), 'rb') as input_file:
        episodes = pickle.load(input_file)
    tests.update({test_name: {
        'info': info,
        'episodes': episodes
    }})
test_types = [test['info']['algorithm'] for _, test in tests.items()]
print("availeble tests:\n {}\ntests_types:\n {}".format(
    list(tests.keys()),
    test_types))

availeble tests:
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]
tests_types:
 ['PG', 'PG', 'PG', 'PG', 'PG', 'PPO', 'PPO', 'PPO', 'PPO', 'PPO', 'A3C', 'A3C', 'A3C', 'A3C', 'A3C', 'A3C', 'A3C', 'A3C', 'A3C', 'A3C', 'IMPALA', 'IMPALA', 'IMPALA', 'IMPALA', 'IMPALA', 'latency_greedy', 'binpacking']


# Necessary stats of a test

In [2]:
final_stats = []
for test_id in range(start_test, end_test):
    users_distances_episodes_average = [episode['users_distances'].mean() for episode in tests[test_id]['episodes']]
    users_distances_test_average = sum(users_distances_episodes_average) / len(users_distances_episodes_average)
    num_consolidated_episodes_average = [episode['num_consolidated'].mean() for episode in tests[test_id]['episodes']]
    num_consolidated_test_average = sum(num_consolidated_episodes_average) / len(num_consolidated_episodes_average)
    # print(users_distances_episodes_average)
    # print(users_distances_test_average)
    # print(num_consolidated_episodes_average)
    # print(num_consolidated_test_average)
    final_stats.append({
        'test_id': test_id,
        # 'checkpoint': tests[test_id]['info']['checkpoint'],
        'network_id': tests[test_id]['info']['network_id'],
        # 'trace_id_test': tests[test_id]['info']['trace_id_test'],
        # 'experiments': tests[test_id]['info']['experiments'],
        # 'penalty_latency': tests[test_id]['info']['penalty_latency'],
        # 'penalalty_consolidated': tests[test_id]['info']['penalty_consolidated'],
        'users_distances_test_average': users_distances_test_average,
        'num_consolidated_test_average': num_consolidated_test_average
    })

final_stats_df = pd.DataFrame(final_stats)
final_stats_df['test_types'] = test_types
# final_stats_df.loc[df['penalalty_consolidated']]

# TODO add the following information to the table (from the info.json file)
# - penatly_latency
# - penalty_conslidation

In [3]:
tests[test_id]['info']

{'dataset_id': 6,
 'workload_id': 0,
 'network_id': 6,
 'trace_id': 2,
 'episode_length': 3453,
 'algorithm': 'binpacking'}

# compare tests - tables

In [4]:
final_stats_df

,test_id,network_id,users_distances_test_average,num_consolidated_test_average,test_types
0,0,6,1.978982,4.934608,PG
1,1,6,2.018397,4.998262,PG
2,2,6,1.982630,5.997625,PG
3,3,6,1.956462,4.998494,PG
4,4,6,1.901134,4.995366,PG
5,5,6,2.124758,2.463365,PPO
6,6,6,1.989454,5.445989,PPO
7,7,6,3.441384,5.989285,PPO
8,8,6,3.509778,4.888908,PPO
9,9,6,2.356147,4.560730,PPO


In [5]:
final_stats_df[(final_stats_df['network_id'] == 1)]

,test_id,network_id,users_distances_test_average,num_consolidated_test_average,test_types


# compare tests - plots

In [6]:
final_stats_df[(final_stats_df['network_id'] == 7) & (final_stats_df['trace_id_test'] == 2)].sort_values(by=['num_consolidated_test_average']) #& (
    # (final_stats_df['experiments'] == 1) | (final_stats_df['experiments'] == 2) | (final_stats_df['experiments'] == 3))]

KeyError: 'trace_id_test'

In [ ]:
final_stats_df[(final_stats_df['network_id'] == 7) & (final_stats_df['trace_id_test'] == 2)].sort_values(by=['users_distances_test_average']) #& (


,test_id,network_id,trace_id_test,experiments,penalty_latency,penalalty_consolidated,users_distances_test_average,num_consolidated_test_average,test_types
32,32,7,2,1,10,2,2.339188,5.640892,PPO
27,27,7,2,0,10,0,2.342369,4.716768,PPO
38,38,7,2,3,2,10,3.173771,0.895337,PPO
39,39,7,2,4,0,10,3.499669,4.999305,PPO
36,36,7,2,2,10,10,4.172751,4.466319,PPO
